## SQL实战 | nowcoder 91题总结
### 第二部分 SQL:21-40题

　　ID：wlong
　　数据分析爱好者、目前小白一枚
　　邮箱：wlong826@163.com
　　和鲸社区：[【wlong】](https://www.heywhale.com/home/user/profile/5f0a61d72be199002c073935)
　　如果有不完善的地方，欢迎小伙伴评论区留言！
　　最近博客、和鲸每周都会有所更新，欢迎大家关注点赞分享哦！
 
 ----
 
 　　这里呢，建议SQL不熟练的小伙伴先去**nowcoder网**上刷一遍，遇到不会的时候再过来看**答案**，总共**80道题**左右，但nowcoder网上有些题目存在**断层不连续**，序号写到了91题，所以这里主要也是以官网的序号为准。刷题链接见这[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　牛客网SQL刷题链接：[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　如果对爬虫感兴趣的小伙伴，看这看这👉：[【零基础入门Python爬虫--豆瓣、58同城、安居客等】](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)

　关于Pandas数据处理的一些小总结，看这看这👉：[【Pandas数据处理120题知识点总结】](https://www.heywhale.com/mw/project/60c0b06919d601001898394c)

　经典数分案例泰坦尼克号的生存分析，看这看这👉：[【经典案例之泰坦尼克号乘客生存情况预测分析 】](https://www.heywhale.com/mw/project/60d18448056f570017c149dc)
 
 　关于Hive SQL的一点小小总结，看这看这👉：[【《Hive 编程指南》的小小总结】](https://www.heywhale.com/mw/project/60f00c93be5f0900176366a6)

　如果小伙伴对于Python实现数据结构感兴趣，可以看这👉：[【线性表】](https://www.heywhale.com/mw/project/60e7b58cbe5f09001761ac87)、[【栈和队列】](https://www.heywhale.com/mw/project/60ebe5de3f2448001705ccc2)

### 21. 查找在职员工自入职以来的薪水涨幅情况(★★★★★)

　　给出在职员工编号emp_no以及其对应的薪水涨幅growth，并按照growth进行升序;

In [ ]:
# 方法一：窗口函数
select
t.emp_no
,(t.last_salary - t.first_salary) growth
from
(
	select
	emp_no
	,first_value(salary) over(partition by emp_no order by from_date) first_salary
	,last_value(salary) over(partition by emp_no order by from_date) last_salary
	,last_value(to_date) over(partition by emp_no order by from_date) last_date
	from salaries
) t
where t.last_date = '9999-01-01'
order by growth;

In [ ]:
# 方法二:连接查询
select
a.emp_no
,b.salary - a.salary growth
from
( 
	select # 先把每个员工刚入职时的编号和薪水找出来
	emp_no
	,salary
	from salaries
	where from_date in (
							select min(from_date)
							from salaries
							group by emp_no
						)
) a
inner join 
(
	select # 把每个在职员工的编号和薪水找出来
	emp_no
	,salary
	from salaries
	where to_date = '9999-01-01'
) b
on a.emp_no = b.emp_no
order by growth;

### 22. 统计各个部门的工资记录数

　　统计各个部门的工资记录数，给出部门编码dept_no、部门名称dept_name以及部门在salaries表里面有多少条记录sum，按照dept_no升序排序；

In [ ]:
select
d.dept_no
,d.dept_name
,count(s.emp_no) sum
from dept_emp d_e
left join departments d
on d_e.dept_no = d.dept_no
right join salaries s
on d_e.emp_no = s.emp_no
group by d.dept_no, d.dept_name
order by d.dept_no;

### 23. 对所有员工的薪水按照salary降序进行1-N的排名(★★★★)

　　对所有员工的薪水按照salary降序进行1-N的排名，要求相同salary并列且按照emp_no升序排列；

In [ ]:
select
emp_no
,salary
,dense_rank() over(order by salary desc) t_rank # 先按照薪资降序排列，给出排名序号
from salaries
order by salary desc, emp_no asc;

### 24. 	获取所有非manager员工当前的薪水情况(★★★★)

　　获取所有非manager员工薪水情况，给出dept_no、emp_no以及salary；

In [ ]:
# 方法一：子查询
select
dept_no
,s.emp_no
,salary
from salaries s
join dept_emp d_e
on s.emp_no = d_e.emp_no
where s.emp_no not in (select emp_no from dept_manager);

In [ ]:
# 方法二：表连接
SELECT
    b.dept_no,
    a.emp_no,
    d.salary
FROM
    employees a,
    dept_emp b,
    dept_manager c,
    salaries d
WHERE
    a.emp_no = b.emp_no
    AND b.dept_no = c.dept_no
    AND b.emp_no != c.emp_no
    AND a.emp_no = d.emp_no;

### 25. 获取员工其当前的薪水比其manager当前薪水还高的相关信息(★★★★★)

　　第一列给出员工的emp_no，
　　第二列给出其manager的manager_no，
　　第三列给出该员工当前的薪水emp_salary,
　　第四列给该员工对应的manager当前的薪水manager_salary

In [ ]:
select
employee.emp_no
,manager.emp_no manager_no
,employee.salary emp_salary
,manager.salary manager_salary
from 
(
	select # 先获取manager薪水
	dept_no
	,d_manager.emp_no
	,salary
	from dept_manager d_manager
	join salaries s
	on d_manager.emp_no = s.emp_no
) manager
join
(
	select
	dept_no
	,d_emp.emp_no
	,salary
	from dept_emp d_emp
	join salaries s 
	on d_emp.emp_no = s.emp_no
	where d_emp.emp_no not in (select emp_no from dept_manager)
) employee
on manager.dept_no = employee.dept_no
where employee.salary > manager.salary;

### 26. 汇总各个部门当前员工的title类型的分配数目(★★★★★)

　　结果给出部门编号dept_no、dept_name、其部门下所有的员工的title以及该类型title对应的数目count，结果按照dept_no升序排序；

In [ ]:
select
d_emp.dept_no
,dept_name
,title
,count(title) count
from dept_emp d_emp
join departments d
on d_emp.dept_no = d.dept_no
join titles t
on d_emp.emp_no = t.emp_no
group by 1,2,3
order by 1;

### 28. 查找描述信息中特定条件电影(★★★★★)

　　查找描述信息(film.description)中包含robot的电影对应的分类名称(category.name)以及电影数目(count(film.film_id))，而且还需要该分类包含电影总数量(count(film_category.category_id))>=5部；

In [ ]:
# 方法一
select
a.name
,count(a.category_id)
from
(
	select # 先找出包含robor的电影分类名称以及电影数量
	c.name
	,c.category_id
	from film_category f_c
	join film f
	on f_c.film_id = f.film_id
	join category c
	on f_c.category_id = c.category_id
	where description like '%robot%' # 条件一：包含robot
) a
join
(
	select # 条件三：计算不同分类下的所有电影数量
	category_id
	,count(f_c.category_id) count2
	from film_category f_c
	group by 1
) b
on a.category_id = b.category_id
where b.count2 >= 5
group by 1;

In [ ]:
# 方法二：
select
c.name
,count(f_c.film_id) count1
from film_category f_c
join film f
on f_c.film_id = f.film_id
join category c
on f_c.category_id = c.category_id
where description like '%robot%' # 条件一：包含robot
and f_c.category_id	in ( # 条件三：计算不同分类下的所有电影数量
							select 
							category_id
							from film_category f_c
							group by 1
							having count(f_c.category_id) >= 5
						);
group by 1 # 条件2：分组求电影数目


### 29. 使用join查询方式找出没有分类的电影id以及名称

In [ ]:
select
f.film_id
,title
from film f
left join film_category f_c
on f.film_id = f_c.film_id
where f_c.category_id is null;

### 30. 使用子查询的方式找出属于Action分类的所有电影对应的title,description

In [ ]:
select
title
,description
from film f
where f.film_id in (
						select film_id
						from category c
						join film_category f_c
						on c.category_id = f_c.category_id
						where c.name = 'Action'
					);

### 32. 将employees表的所有员工的last_name和first_name拼接起来作为Name，中间以一个空格区分

In [ ]:
select concat(last_name, ' ', first_name) Name
from employees；

### 33. 创建一个actor表，包含如下列信息

In [ ]:
create table actor(
	actor_id smallint(5) primary key,
	first_name varchar(45) not null,
	last_name varchar(45) not null,
	last_update date not null
);

### 34. 批量插入数据

　　对于表actor批量插入如下数据(不能有2条insert语句哦!)；

In [ ]:
insert into actor (
					actor_id, 
					first_name, 
					last_name, 
					last_update
				  )
values (1, 'PENELOPE', 'GUINESS', '2006-02-15 12:34:33'),
	   (2, 'NICK', 'WAHLBERG', '2006-02-15 12:34:33');

### 35. 批量插入数据，不使用replace操作

　　mysql中常用的三种插入数据的语句:
　（1）insert into表示插入数据，数据库会检查主键，如果出现重复会报错；
　（2）replace into表示插入替换数据，需求表中有PrimaryKey，或者unique索引，如果数据库已经存在数据，则用新数据替换，如果没有数据效果则和insert into一样；
　（3）insert ignore表示，如果中已经存在相同的记录，则忽略当前新数据；

In [ ]:
insert ignore into actor values("3","ED","CHASE","2006-02-15 12:34:33");

### 36. 创建一个actor_name表

　　创建一个actor_name表，并且将actor表中的所有first_name以及last_name导入该表；

In [ ]:
#(1)常规创建
create table if not exists 目标表
#(2)复制表格
create 目标表 like 来源表
#(3)将table1的部分拿来创建table2
create table if not exists actor_name
(
first_name varchar(45) not null,
last_name varchar(45) not null
)
select first_name,last_name
from actor;
# (4)创建跟插入分开
CREATE TABLE actor_name    -- 创建表
(first_name varchar(45) NOT NULL,
 last_name varchar(45) NOT NULL);

INSERT INTO actor_name  -- 插入数据
SELECT first_name,last_name
FROM actor;

In [ ]:
# 方法三：参考答案里，自连接
select s.emp_no, s.salary, e.last_name, e.first_name
from salaries s join employees e
on s.emp_no = e.emp_no
where s.salary = (
				    select s1.salary
				    from salaries s1 join salaries s2      -- 自连接查询
				    on s1.salary <= s2.salary
				    group by s1.salary                     -- 当s1<=s2链接并以s1.salary分组时一个s1会对应多个s2
				    having count(distinct s2.salary) = 2   -- (去重之后的数量就是对应的名次)
				  )

### 37. 创建唯一索引和普通索引

　　对first_name创建唯一索引uniq_idx_firstname，对last_name创建普通索引idx_lastname；

In [ ]:
# 唯一索引
alter table actor add unique uniq_idx_firstname(first_name);
# 普通索引
alter table actor add index idx_lastname(last_name);
# 或者
create index idx_lastname on actor(last_name);

　（1）添加主键，该语句添加一个主键，这意味着索引值必须是唯一的，且不能为NULL。
　　ALTER TABLE tal_name ADD PRIMARY KEY (col_list);

　（2）添加唯一索引，这条语句创建索引的值必须是唯一的。
　　ALTER TABLE tbl_name ADD UNIQUE index_name (col_list);

　（3）添加普通索引，索引值可出现多次
　　ALTER TABLE tbl_name ADD INDEX index_name (col_list);

　（4）添加全文索引，该语句指定了索引为 FULLTEXT ，用于全文索引。
　　ALTER TABLE tbl_name ADD FULLTEXT index_name (col_list);

　（5） 删除索引
　　DROP INDEX index_name ON tbl_name; 
　　// 或者
　　ALTER TABLE tbl_name DROP INDEX index_name；
　　ALTER TABLE tbl_name DROP PRIMARY KEY;

### 38. 针对actor表创建视图actor_name_view

　　针对actor表创建视图actor_name_view，只包含first_name以及last_name两列，并对这两列重新命名，first_name为first_name_v，last_name修改为last_name_v；

In [ ]:
create view actor_name_view as
select 
first_name first_name_v
,last_name last_name_v
from actor;

### 39.针对salaries表emp_no字段创建索引idx_emp_no，查询emp_no为10005，使用强制索引

In [ ]:
select *
from salaries
force index (idx_emp_no)
where emp_no = 10005;
# https://www.yangdx.com/2020/05/151.html

### 40. 在last_update后面新增加一列名字为create_date

　　现在在last_update后面新增加一列名字为create_date, 类型为datetime, NOT NULL，默认值为'2020-10-01 00:00:00'；

In [ ]:
alter table actor add(create_date datetime not null default '2020-10-01 00:00:00');